In [1]:
!pip install pyLDAvis

In [2]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.gensim
import pyLDAvis.gensim_models
import pyLDAvis

In [3]:
#데이터 불러오기

token_df=pd.DataFrame()
token_df=pd.read_csv(r"C:\Users\hojun\project\token_data\TOTAL_token_kr_song_chart_data.csv")
print(token_df['lyric_tokens'].isna().sum())
token_df['lyric_tokens'] = token_df['lyric_tokens'].fillna('')


15


In [4]:
token_df.head()

,song_year,song_rank,singer,title,Genre,lyric,lyric_tokens
0,2000,1,조성모,아시나요,발라드,아시나요 얼마나 사랑했는지\n그댈 보면 자꾸 눈물이 나서\n차마 그대 바라보지 못하...,한참 살아왔던거죠 눈물 힘 흔적들 길목 그댄 외면 병 때문 채 몇 하늘 마지막 뒷모...
1,2000,2,조규만,다 줄거야 (Acoustic Ver.),발라드,그대 내게 다가오는 그 모습\n자꾸 다시 볼수 없을 것만 같아서\n감은 두눈 뜨지 ...,힘 모습 내품 사랑을 지쳐있던거야 못한거야 조금씩 그동안 눈물속 감사할뿐야 두눈 날...
2,2000,3,DJ DOC,Run To You,랩/힙합,하나 투 원 투 쓰리 포\nBOUNCE with me BOUNCE\nwith me\...,원하는걸 외로울땐 허락 니맘 시간 포 니안 원한다는걸 하늘 너와 원 시선 무엇 문 ...
3,2000,4,god,거짓말,발라드,미안해\n난 니가 싫어졌어 우리\n이만 헤어져\n다른 여자가 생겼어 너보다\n훨씬 ...,이러 걱정 속일 정신 화낼거야 어디 이상 고생 거짓말 행복 여자 저리 놈 정리 몇 ...
4,2000,5,클론,초련(初戀) (Techno Mix) (Feat. 윤진),댄스,난 그냥 좋았어 니 앞에만 서면\n요즘 내가 왜 이러는지 나도 몰라\n니 생각만 하...,첨 샤워 전화 시간씩 조차 사소 눈 쇼핑 설명 비상금 카페 거울 주체 햇살 틈 오레...


In [5]:
#count_vectorizer 학습
#가사 정보 벡터화


# 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
# bigram도 포함
vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))

data_vectorized = vectorizer.fit_transform(token_df['lyric_tokens'])
print('CountVectorizer Shape:', data_vectorized.shape)

CountVectorizer Shape: (2400, 1000)


In [6]:
#LDA 모델 불러와서 학습?

# lda_model = LatentDirichletAllocation(n_components=5, random_state=42)

#파라미터 설정
lda_model = LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

#model 학
lda_model.fit(data_vectorized)

# 주제별 단어 출력
for idx, topic in enumerate(lda_model.components_):
    print(f"Topic {idx}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])


Topic 0:
['만큼', '오늘 사랑', '마지막', '이유', '그날', '그땐', '그리움', '후회', '날들', '상처']
Topic 1:
['걱정', '내겐', '마지막', '후회', '여자', '언제', '때문', '필요', '모두', '남자']
Topic 2:
['심장', '처음', '남자', '입술', '가슴 사랑', '눈빛', '전부', '그댄', '운명', '지금']
Topic 3:
['노래', '기분', '거리', '지금', '가지', '전화', '음악', '영원', '친구', '아침']
Topic 4:
['머리', '시작', '여자', '이상', '필요', '눈빛', '표정', '여기', '정신', '아무']


In [7]:
# 토픽의 주제를 다음과 같이 임의로 정리함
topics = [
    "관계와 후회",         # Topic 0
    "고독과 그리움",       # Topic 1
    "불안과 거짓",         # Topic 2
    "삶과 감정",           # Topic 3
    "사랑의 끝과 소중함"   # Topic 4
]

In [8]:
#토픽 단어 빈도를 gensim을 이용해 시각화

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import gensim
from gensim import corpora


text_data= [i.split(" ") for i in token_df['lyric_tokens']]

# 사전과 말뭉치 생성
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

# gensim LDA 모델 학습
gensim_lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)

# pyLDAvis 시각화
lda_vis = gensimvis.prepare(gensim_lda_model, corpus, dictionary)
pyLDAvis.display(lda_vis)


In [9]:
#LDA 결과 df 저장

#변환
topic_list = lda_model.transform(data_vectorized)
 
lyric_topic_list = []
lyric_topic_pr_list = []

#리스트로 일단 저장하고 Df에 넣어줌
for n in range(topic_list.shape[0]):
    topic_index = topic_list[n].argmax()
    topic_pr = topic_list[n].max()
    lyric_topic_list.append(topic_index)
    lyric_topic_pr_list.append(topic_pr)
    
token_df['topic'] = lyric_topic_list
token_df['topic_pr'] = lyric_topic_pr_list




In [11]:
token_df.to_csv(r"C:\Users\hojun\2024_10_10_project\data\topic_data\topic_total_data.csv",index=False)

In [13]:

token_df = token_df.drop_duplicates(subset=['title'])

#토픽별로 topic_pr이 가장 높은 3곡으로 보여줌
for topic in range(len(token_df['topic'].unique())):
    print('Topic #', topics[topic], topic, '-----------------------------')
    top_pr_topics = token_df[token_df['topic'] == topic].sort_values(by='topic_pr', ascending=False)
    print(top_pr_topics['title'].iloc[0])
    print(top_pr_topics['title'].iloc[1])
    print(top_pr_topics['title'].iloc[2], '\n')


Topic # 관계와 후회 0 -----------------------------
연애소설 (Feat. 아이유)
Suddenly
...사랑했잖아... 

Topic # 고독과 그리움 1 -----------------------------
내 입술... 따뜻한 커피처럼
이별이 온다
취중고백 

Topic # 불안과 거짓 2 -----------------------------
다정히 내 이름을 부르면
나란 놈은 답은 너다 (Feat. 하림)
CALL ME BABY 

Topic # 삶과 감정 3 -----------------------------
한여름밤의 꿀
기다렸다 가
빌었어 

Topic # 사랑의 끝과 소중함 4 -----------------------------
자니 (Feat. Dynamic Duo)
화장 지웠어 (Feat. Zion.T, HA:TFELT) (No Make Up)
요즘것들 (Feat. ZICO, DEAN) 

